In [1]:
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
tsla = yf.Ticker("tsla")

tsla_hist = tsla.history(period='5y', interval='1d', end='2025-06-13')
# tsla_hist

In [3]:
tsla_hist.drop(columns=['Dividends', 'Stock Splits'], inplace=True)

In [4]:
tsla_hist.index =pd.to_numeric(tsla_hist.index)
tsla_hist.index = tsla_hist.index/(max(tsla_hist.index)) + 1

In [5]:
for col in tsla_hist.columns:
    tsla_hist[col] = tsla_hist[col]/max(tsla_hist[col]) + 1

In [6]:
X = tsla_hist.drop(columns=['Close'])
y = tsla_hist['Close']

In [7]:
X['Date'] = X.index

In [8]:
for delay in range(1,4):
    X[f"Delay {delay}"] = y.iloc[(3-delay):-(delay)]

In [9]:
X_sample = [[d, h, l, o, vol, v1, v2, v3] for d, h, l, o, vol, v1, v2, v3 in zip(X['Date'].iloc[3:].values, X['High'].iloc[3:].values, X['Low'].iloc[3:].values,X['Open'].iloc[3:].values, X['Volume'].iloc[3:].values, X['Delay 1'].dropna().values, X['Delay 2'].dropna().values, X['Delay 3'].dropna().values)]
y_sample = y.iloc[3:].values

In [27]:
X_df = pd.DataFrame(X_sample)

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
print(X_df)

In [10]:
len(X['Date'].iloc[3:].values), len(X_sample)

(1252, 1252)

In [11]:
goods = []
for samples in X_sample:
    for sample in samples:
        if np.isnan(sample):
            print("contains nan")
        elif sample ==0:
            print("contains 0")
        elif sample != None:
            goods.append(sample)

In [12]:
len(goods)

10016

In [13]:
len(y_sample), len(X_sample)

(1252, 1252)

In [14]:
torch.set_default_dtype(torch.float64)

In [15]:
X = torch.from_numpy(np.array(X))
y = torch.from_numpy(np.array(y))

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [17]:
len(X_test)

251

In [18]:
train_data = [(X,y) for X, y in zip(X[:-273], y[:-273])]
test_data = [(X,y) for X, y in zip(X[-273:], y[-273:])]

In [19]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloaader= DataLoader(test_data, batch_size=32, shuffle=True)

In [26]:
count = 0
batch = 0
for (X, y) in train_dataloader:
    if (torch.any(X.isnan())):
        print(f"Found on batch: {batch}")
        count+=1
    batch +=1


Found on batch: 1
Found on batch: 29


In [21]:
class MLP(nn.Module):
    def __init__(self, input_features, hidden_features, output_features):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(input_features, hidden_features),
            nn.ReLU()
        )
        self.l2 = nn.Sequential(
            nn.Linear(hidden_features, output_features),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        return x

In [22]:
mlpmodel = MLP(8, 15, 1)

In [23]:
loss_fn = nn.MSELoss()

optimizer = torch.optim.SGD(params=mlpmodel.parameters(), lr=0.0001)

In [24]:
EPOCHS = 1

mlpmodel.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for batch, (X, y) in enumerate(train_dataloader):
        preds = mlpmodel(X)
        # print(f"Preds + {preds}")
        loss = loss_fn(torch.unsqueeze(y, 1), preds)
        total_loss+= loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"We're on batch: {batch} + Loss: {total_loss/batch}")


We're on batch: 0 + Loss: inf
We're on batch: 1 + Loss: 2.257965103049022
We're on batch: 2 + Loss: 1.666181850929313
We're on batch: 3 + Loss: 1.4782561260499947
We're on batch: 4 + Loss: 1.3791924458454652
We're on batch: 5 + Loss: 1.308832701773658
We're on batch: 6 + Loss: 1.2736194402461434
We're on batch: 7 + Loss: 1.2353050065706124
We're on batch: 8 + Loss: nan
We're on batch: 9 + Loss: nan
We're on batch: 10 + Loss: nan
We're on batch: 11 + Loss: nan
We're on batch: 12 + Loss: nan
We're on batch: 13 + Loss: nan
We're on batch: 14 + Loss: nan
We're on batch: 15 + Loss: nan
We're on batch: 16 + Loss: nan
We're on batch: 17 + Loss: nan
We're on batch: 18 + Loss: nan
We're on batch: 19 + Loss: nan
We're on batch: 20 + Loss: nan
We're on batch: 21 + Loss: nan
We're on batch: 22 + Loss: nan
We're on batch: 23 + Loss: nan
We're on batch: 24 + Loss: nan
We're on batch: 25 + Loss: nan
We're on batch: 26 + Loss: nan
We're on batch: 27 + Loss: nan
We're on batch: 28 + Loss: nan
We're on 